In [2]:
#library
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.applications as ap

In [3]:
#mount file from google drive
from google.colab import drive
drive.mount('/content/drive')
#grab the data
img512 = np.load('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/img512.npy')
label  = np.load('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/label.npy')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
X512_train = img512
y512_train = label
y512_train = tf.keras.utils.to_categorical(y512_train, 4)

In [5]:
#checkpoint
checkpoint_filepath = '/content/drive/MyDrive/Colab Notebooks/FINAL_DL/checkpoint/resnet'
model_checkpoint_callack = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weight_only = True,
    save_freq = 'epoch',
    mode = 'auto',
    save_best_only = True,
    monitor = 'val_accuracy'
)

In [6]:
#resnet
def model_resnet(X_train, y_train,epochs = 8):
    base_model = ap.ResNet50(weights='imagenet',input_shape=(512,512,3),include_top=False)
    base_model.trainable = False

    head_model = base_model.output
    head_model = keras.layers.Flatten()(head_model)
    #head_model = keras.layers.Dense(512, activation = 'relu')(head_model)
    #head_model = keras.layers.Dropout(0.5)(head_model)
    #head_model = keras.layers.Dense(128, activation = 'relu')(head_model)
    head_model = keras.layers.Dense(4,activation='softmax')(head_model)

    model = keras.Model(base_model.input,head_model)
    #model = base_model

    callbacks = [keras.callbacks.TensorBoard(log_dir='/content/drive/MyDrive/Colab Notebooks/FINAL_DL/logs/resnet'),model_checkpoint_callack]
    
    model.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy',metrics=['accuracy'])
    model.fit(X_train,y_train,batch_size=128, epochs=epochs, callbacks = callbacks, validation_split = 0.2)

    base_model.trainable = True
    model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='categorical_crossentropy',metrics=['accuracy'])
    history = model.fit(X_train,y_train,batch_size=128, epochs=epochs, callbacks = callbacks, validation_split = 0.2)
    return model,history

In [ ]:
model_resnet_m2,history= model_resnet(X512_train,y512_train,4)

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/4
19/19 [==============================] - ETA: 0s - loss: 38.2326 - accuracy: 0.6267INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/FINAL_DL/checkpoint/resnet/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


19/19 [==============================] - 260s 14s/step - loss: 38.2326 - accuracy: 0.6267 - val_loss: 12.3095 - val_accuracy: 0.8183
Epoch 2/4
19/19 [==============================] - ETA: 0s - loss: 5.1132 - accuracy: 0.9004INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/FINAL_DL/checkpoint/resnet/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


19/19 [==============================] - 261s 14s/step - loss: 5.1132 - accuracy: 0.9004 - val_loss: 5.9892 - val_accuracy: 0.9067
Epoch 3/4
19/19 [==============================] - ETA: 0s - loss: 0.9124 - accuracy: 0.9692 INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/FINAL_DL/checkpoint/resnet/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


19/19 [==============================] - 266s 14s/step - loss: 0.9124 - accuracy: 0.9692 - val_loss: 4.6097 - val_accuracy: 0.9133
Epoch 4/4
19/19 [==============================] - 244s 13s/step - loss: 0.2599 - accuracy: 0.9837 - val_loss: 4.9711 - val_accuracy: 0.9000
Epoch 1/4


In [ ]:
model_resnet_m2.save_weights('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/weights/resnet_m2_weights.h5')

In [ ]:
#save the curve
his = pd.DataFrame(history.history)
his.to_pickle('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/curve/history_resnet2.pkl')

In [ ]:
model_resnet_m2.summary()